In [6]:
import pandas as pd
import numpy as np
from scipy.spatial.distance import euclidean
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, accuracy_score

dataset loading

In [7]:
df = pd.read_csv("BOM.csv")
print(df.head())
dataset_shape = df.shape
print(f"Dataset shape: {dataset_shape}")

         Date Location  MinTemp  MaxTemp  Rainfall  Evaporation  Sunshine  \
0  2008-12-01   Albury     13.4     22.9       0.6          NaN       NaN   
1  2008-12-02   Albury      7.4     25.1       0.0          NaN       NaN   
2  2008-12-03   Albury     12.9     25.7       0.0          NaN       NaN   
3  2008-12-04   Albury      9.2     28.0       0.0          NaN       NaN   
4  2008-12-05   Albury     17.5     32.3       1.0          NaN       NaN   

  WindGustDir  WindGustSpeed WindDir9am  ... Humidity9am  Humidity3pm  \
0           W           44.0          W  ...        71.0         22.0   
1         WNW           44.0        NNW  ...        44.0         25.0   
2         WSW           46.0          W  ...        38.0         30.0   
3          NE           24.0         SE  ...        45.0         16.0   
4           W           41.0        ENE  ...        82.0         33.0   

   Pressure9am  Pressure3pm  Cloud9am  Cloud3pm  Temp9am  Temp3pm  RainToday  \
0       1007.7    

remove invalid targets

In [8]:
df_filtered = df[df['RainTomorrow'].isin(['Yes', 'No'])]
dataset_shape = df_filtered.shape
print(f"Dataset shape: {dataset_shape}")
df_filtered.to_csv("cleaned_target.csv", index=False)
df = pd.read_csv("cleaned_target.csv")
print(df.shape)

Dataset shape: (142193, 23)
(142193, 23)


 در این بخش همانطور که در صورت سوال خواسته شده، داده هایی را که در هدف مقداری غیر از بله یا خیی داشتند،حذف کرده ایم.همچنین در اخر اندازه دیتافریم را بررس کردم تا مطمين شوم عملیات حذف اتفاق افتاده است. همچنین دیتافریم بدست آمده را در یک فایل جدید ذخیره کردم

Add month to data set

In [9]:
df['Date'] = pd.to_datetime(df['Date'])
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
print(df.head())
print(df.shape)

        Date Location  MinTemp  MaxTemp  Rainfall  Evaporation  Sunshine  \
0 2008-12-01   Albury     13.4     22.9       0.6          NaN       NaN   
1 2008-12-02   Albury      7.4     25.1       0.0          NaN       NaN   
2 2008-12-03   Albury     12.9     25.7       0.0          NaN       NaN   
3 2008-12-04   Albury      9.2     28.0       0.0          NaN       NaN   
4 2008-12-05   Albury     17.5     32.3       1.0          NaN       NaN   

  WindGustDir  WindGustSpeed WindDir9am  ... Pressure9am  Pressure3pm  \
0           W           44.0          W  ...      1007.7       1007.1   
1         WNW           44.0        NNW  ...      1010.6       1007.8   
2         WSW           46.0          W  ...      1007.6       1008.7   
3          NE           24.0         SE  ...      1017.6       1012.8   
4           W           41.0        ENE  ...      1010.8       1006.0   

   Cloud9am  Cloud3pm  Temp9am  Temp3pm  RainToday  RainTomorrow  Year  Month  
0       8.0       NaN   

در این بخش از تاریخ های موجود در دیتاست اصتفاده کردم و ماه و سال را از آنها استخراج کردم. این کار برای ادامه کد که میخواهیم از میانگین و مد برای داده های خالی استفاده کنیم نیاز است

Median and Mode

In [10]:

#columns with numerical and object values
numeric_columns = df.select_dtypes(include=['float64', 'int64']).columns
object_columns = df.select_dtypes(include=['object']).columns

# Function to replace missing values in numerical columns with the median of the corresponding month
def fill_numeric_with_monthly_median(dataframe, columns):
    for column in columns:
        # Compute the median for each year-month group
        medians = dataframe.groupby(['Year', 'Month'])[column].transform('median')
        # Replace empty strings with NaN if they exist
        dataframe[column].replace("", pd.NA, inplace=True)
        # Fill missing values with the corresponding median
        dataframe[column].fillna(medians, inplace=True)
    return dataframe

# Function to replace missing values in object columns with the mode (most frequent value) of the corresponding month
def fill_object_with_monthly_mode(dataframe, columns):
    for column in columns:
        # Compute the mode for each year-month group
        modes = dataframe.groupby(['Year', 'Month'])[column].transform(lambda x: x.mode()[0] if not x.mode().empty else x)
        # Replace empty strings with NaN if they exist
        dataframe[column].replace("", pd.NA, inplace=True)
        # Fill missing values with the corresponding mode
        dataframe[column].fillna(modes, inplace=True)
    return dataframe

# Replace missing values with the median for numerical columns
df = fill_numeric_with_monthly_median(df, numeric_columns)

# Replace missing values with the mode for object columns
df = fill_object_with_monthly_mode(df, object_columns)

# Drop the 'Year' and 'Month' columns as they were temporary for the calculation
df.drop(columns=['Year', 'Month'], inplace=True)

# Display the first few rows of the DataFrame to verify
print(df.shape)
# Save the cleaned DataFrame back to a CSV file (optional)
df.to_csv('NotNull_dataset.csv', index=False)


(142193, 23)


در این بخش داده های عددی و رشته ای را جداگانه بررسی کردم . داده های عددی ای که مقدار نداشتند را با میانگین داده های دیگر در همان ماه پر کردم . داده های رشته ای که مقدار نداشتند را با ماکسیمم داده تکرار شده در همان ماه یا همان مد داده ها ، پر کردم

Reduce the number of variables

In [11]:
# Calculate the correlation matrix
correlation_matrix = df.corr().abs()

# Get the upper triangle of the correlation matrix
upper_triangle = correlation_matrix.where(np.triu(np.ones(correlation_matrix.shape), k=1).astype(bool))

# Find feature pairs with correlation coefficients above 0.95
highly_correlated_pairs = [(i, j) for i in range(len(upper_triangle.columns)) for j in range(i+1, len(upper_triangle.columns)) if upper_triangle.iloc[i, j] > 0.95]

# Print highly correlated feature pairs
print("Highly Correlated Feature Pairs:")
for pair in highly_correlated_pairs:
    print(df.columns[pair[0]], "-", df.columns[pair[1]], ": Correlation =", upper_triangle.iloc[pair[0], pair[1]])
    
removed_features = []
for pair in highly_correlated_pairs:
    removed_features.append(df.columns[pair[1]])
    df.drop(columns=[df.columns[pair[1]]], inplace=True)
print("Removed features:", removed_features)

print(df.shape)
df.to_csv('reduced_dataset.csv', index=False)


/var/folders/5l/n23dh2ds1wn5534zd6s5hkfm0000gn/T/ipykernel_10802/3906929220.py:2: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  correlation_matrix = df.corr().abs()


Highly Correlated Feature Pairs:
Location - Pressure9am : Correlation = 0.9716302291283201
WindDir3pm - WindSpeed9am : Correlation = 0.9611560115076542
Removed features: ['Pressure9am', 'WindSpeed9am']
(142193, 21)


در این بخش میخواهیم برای اینکه بتوانیم مدل بهتری ترین کنیم ، تعداد فیچر هارا کم کنیم. یک راه حل این است که میزان همبستگی هر دو فیچر متمایز از همدیگر را بدست آوریم. در صورت سوال گفته شده اگر کوریلیشن بالاتر از ۰.۹۵ بود از هر پیر یکی از فیچر ها حذف شود که در اینجا دو زوج ویژگی با کوریلیشن بیشتر از ۰.۹۵ بدست آوردیم و به صورت دلخواه از هر کدام دومین ویژگی را حذف کردیم. به 
کمک این روش توانستیم تعداد فیچر هارا کم کنیم


----------------------------------------------------------------------------------------------------
problem 3 - question A - answer:

همچنین در انتهای سوال مطرح شده است که آیا نگهداشتن دو فیچر در دیتاست که همبستگی بسیار بالایی با یکدیگر دارند کار درست یا خیر ؟
   از نظر من این کار درست نیست زیرا میتواند مشکلات زیر را ایجاد کند
   برازش بیش‌ازحد: گنجاندن ویژگی‌های بسیار مرتبط در یک مدل می‌تواند منجر به برازش بیش از حد شود، که در آن مدل نویز را در داده‌های آموزشی به جای روابط زیربنایی یاد می‌گیرد

   ناپایداری تخمین های ضرایب: چند خطی بودن می تواند باعث شود که تخمین ضرایب ناپایدار باشد و نسبت به تغییرات کوچک در داده ها بسیار حساس باشد

   تفسیرپذیری کاهش یافته: تفسیر اثرات فردی ویژگی های همبسته روی متغیر هدف زمانی که آنها به شدت هم خطی هستند دشوار می شود

   بنابراین بهتر است یکی از این زوج ویژگی هارا حذف کنیم

Encoding

In [12]:
df = pd.read_csv('reduced_dataset.csv')

# Drop any columns that are not useful or cannot be converted to numerical format
# For example, dropping 'Date' column
df.drop(columns=['Date'], inplace=True)
print(df.shape)

# Separate features (X) and target variable (y)
X = df.drop(columns=['RainTomorrow'])  
y = df['RainTomorrow']
print(X.shape)

# Initialize LabelEncoder for target variable
label_encoder = LabelEncoder()

# Apply label encoding to target variable (boolean)
y = label_encoder.fit_transform(y)

# Apply label encoding to categorical features
for col in X.select_dtypes(include=['object']).columns:
    try:
        X[col] = label_encoder.fit_transform(X[col])
    except Exception as e:
        print(f"Error occurred while encoding column '{col}': {e}")

# Save the DataFrame to a new CSV file
X['Target'] = y
X.to_csv('encoded_data.csv', index=False)


(142193, 20)
(142193, 19)


در این بخش از یک روش انکدینگ استفاده کردیم تا داده های استرینگ را به عدد تبدیل کنیم تا در ادامه برای ترین کردن مدل ها به مشکل نخوریم
من سعی کردم بدون انکدینگ هم مدل را پیاده سازی کنم اما تنها راه ممکن حذف کردن داده های غیر عددی بود که پس از انجام این کار دقت مدل به شدت پایین تر از دقت فعلی بود . برای همین تصمیم گرفتم به جای حذف داد ها از انکد کردن آنها استفاده کنم

10 best features

In [13]:
df = pd.read_csv('encoded_data.csv')
X = df.drop(columns=['Target'])  
y = df['Target']
# Split the data into 'train', 'validation', and 'test' sets
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42)
X_validation, X_test, y_validation, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Initialize a random forest classifier with default hyperparameters
clf = RandomForestClassifier(n_estimators=5, max_depth=10, random_state=42)

# Initialize Sequential Feature Selector
sfs = SequentialFeatureSelector(clf, n_features_to_select=10, direction='forward', scoring='accuracy', cv=4)

# Fit the Sequential Feature Selector to the training data
sfs.fit(X_train, y_train)

# Get the names of the selected features
selected_features = X_train.columns[sfs.support_]

# Print the selected features
print("The 10 best features selected by Sequential Feature Selection:")
for feature in selected_features:
    print(feature)
    
# Assuming 'important_features' contains the names of the ten most important features
important_features = selected_features



# Create a new DataFrame with only the important features and the target column
new_df = df[important_features].copy()
new_df['Target'] = y
new_df.to_csv('important.csv', index=False)


# Optionally, print the sizes of the resulting sets
print("\nTrain set size:", X_train.shape[0])
print("Validation set size:", X_validation.shape[0])
print("Test set size:", X_test.shape[0])

The 10 best features selected by Sequential Feature Selection:
Location
MinTemp
MaxTemp
Sunshine
WindGustSpeed
WindSpeed3pm
Humidity9am
Humidity3pm
Pressure3pm
Temp9am

Train set size: 113754
Validation set size: 14219
Test set size: 14220


در این بخش به کمک کتابخانه رندوم فارست ۱۰ فیچر با اهمیت بیشتر را انتخاب کردم تا مدل بعدی را بر روی دیتاستی با همین ۱۰ ویژگی ترین کنم. این کار قطعا باعث کم شدن دقت نسبت به حالتی که مدل را بر روی همه ویژگی ها ترین کنیم، میشود. اما باعث بالاتر رفتن  سرعت ران شدن کد، میشود. همچنین من کد را بدون حذف ویژگی ها با کمک گوگل کولب ران کردم و دقت بدست آمده با دقت حالتی که در ادامه بررسی میکنیم تفاوت زیادی نداشت 

KNN

In [14]:
df = pd.read_csv('important.csv')
X = df.drop(columns=['Target']) 
y = df['Target']
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42)
X_validation, X_test, y_validation, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)


best_accuracy = 0
best_k = None

# Iterate over different values of k
for k in range(1,100):
    # Initialize K-NN classifier with the current value of k
    knn = KNeighborsClassifier(n_neighbors=k)
    
    # Train the K-NN classifier on the training set
    knn.fit(X_train, y_train)
    
    # Predict labels for the validation set
    y_pred = knn.predict(X_validation)
    
    # Calculate accuracy on the validation set
    accuracy = accuracy_score(y_validation, y_pred)
    
    # Print the accuracy for the current value of k
    print(f'Validation accuracy for k={k}: {accuracy}')
    
    # Check if this k value gives better accuracy than the previous best
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_k = k

# Select the value of k that gives the best performance on the validation set
print(f'\nBest value of k: {best_k} (Validation accuracy: {best_accuracy})')

# Train the selected K-NN model with the best value of k using only the training set
best_knn = KNeighborsClassifier(n_neighbors=best_k)
best_knn.fit(X_train, y_train)

# Evaluate the performance of the selected model on the test set
y_test_pred = best_knn.predict(X_test)
test_accuracy = accuracy_score(y_test, y_test_pred)
print(f'\nAccuracy on the test set: {test_accuracy}')


Validation accuracy for k=1: 0.7935860468387369
Validation accuracy for k=2: 0.8212251213165482
Validation accuracy for k=3: 0.8250228567409804
Validation accuracy for k=4: 0.8321963569871299
Validation accuracy for k=5: 0.8338842393979886
Validation accuracy for k=6: 0.8364160630142766
Validation accuracy for k=7: 0.8371896757859203
Validation accuracy for k=8: 0.8417610239819959
Validation accuracy for k=9: 0.8411983965117097
Validation accuracy for k=10: 0.841128068077924
Validation accuracy for k=11: 0.8409874112103524
Validation accuracy for k=12: 0.8421829945847106
Validation accuracy for k=13: 0.8426049651874252
Validation accuracy for k=14: 0.8426049651874252
Validation accuracy for k=15: 0.8426049651874252
Validation accuracy for k=16: 0.8452071172374991
Validation accuracy for k=17: 0.8440115338631409
Validation accuracy for k=18: 0.8442928475982839
Validation accuracy for k=19: 0.8445038328996414
Validation accuracy for k=20: 0.8440115338631409
Validation accuracy for k=21: 

در اینجا دفت های مختلف الگوریتم در ازای مقادیر مختلف پارامتر کا بدست آمد . بر اساس بهترین کا، بهترین مدل را ترین میکنیم که دقت این مدل در خروجی بالا قابل مشاهده است

-----------------------------------------------------------------------------
problem 3- question e - answer :

yes, it is possible to determine the probability of rainfall using a similar strategy as outlined in the provided code. Here's how you can modify the approach to predict the probability of rainfall:

Model Selection: Choose a model that can provide probability estimates directly. Logistic Regression is a common choice for binary classification tasks where the output can be interpreted as the probability of the positive class (rainfall). Alternatively, ensemble methods like Random Forests and Gradient Boosting Machines (e.g., XGBoost, LightGBM) can also provide probability estimates when configured appropriately.

Training and Prediction: Train the selected model on the training data as before. However, during prediction, instead of using predict() method, use predict_proba() method to obtain the probability estimates for each class (in this case, probability of rainfall and probability of no rainfall).

Evaluation: Evaluate the model's performance using appropriate metrics for probability estimates. Common metrics include log loss (or cross-entropy loss), area under the receiver operating characteristic curve (ROC-AUC), and calibration plots.

Calibration: Optionally, you can calibrate the predicted probabilities to improve their reliability. Calibration ensures that the predicted probabilities are well-calibrated and reflect the true likelihood of rainfall. Techniques such as Platt scaling or Isotonic regression can be used for calibration.

Threshold Selection: Determine an appropriate threshold for converting probability estimates into binary predictions (rainfall or no rainfall). This threshold can be chosen based on the specific requirements of the application, such as minimizing false positives or false negatives.

By following these steps, you can adapt the provided strategy to predict the probability of rainfall rather than just the binary outcome. This allows for a more nuanced understanding of the likelihood of rainfall based on the given features and can be useful for various applications such as risk assessment, resource planning, and decision-making.









RASHT

In [15]:
Rasht_data = pd.read_csv("Rasht.csv")
Rasht_data['Date'] = pd.to_datetime(Rasht_data['Date'])
Rasht_data['Month'] = Rasht_data['Date'].dt.month
Rasht_data = Rasht_data.drop(columns=["Date"])

X = Rasht_data.drop(columns=['RainTomorrow'])  
y = Rasht_data['RainTomorrow']
print(X.shape)

# Initialize LabelEncoder for target variable
label_encoder = LabelEncoder()

# Apply label encoding to target variable (boolean)
y = label_encoder.fit_transform(y)

# Apply label encoding to categorical features
for col in X.select_dtypes(include=['object']).columns:
    try:
        X[col] = label_encoder.fit_transform(X[col])
    except Exception as e:
        print(f"Error occurred while encoding column '{col}': {e}")

# Save the DataFrame to a new CSV file
X['Target'] = y
X.to_csv('rasht_encoded.csv', index=False)


# KNN
df = pd.read_csv('rasht_encoded.csv')
df_2=pd.read_csv('important.csv')
# Extract feature names from both datasets
features_A = set(df.columns)
features_B = set(df_2.columns)

# Find common features between dataset A and dataset B
common_features = features_A.intersection(features_B)

# Select only the common features from dataset B
df_selected = df[common_features]
print(df_selected.shape)

X = df_selected.drop(columns=['Target']) 
y = df_selected['Target']
print(X.shape)
print(type(X))
print(X.flags)


best_knn.fit(X,y)
y_pred = best_knn.predict(X)


# Calculate and print the accuracy on the test set
report = classification_report(y, y_pred)
print(report)





(2, 22)
(2, 11)
(2, 10)
<class 'pandas.core.frame.DataFrame'>
<Flags(allows_duplicate_labels=True)>


/var/folders/5l/n23dh2ds1wn5534zd6s5hkfm0000gn/T/ipykernel_10802/3952886121.py:39: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  df_selected = df[common_features]


AttributeError: 'Flags' object has no attribute 'c_contiguous'

در این بخش دیتاست شهر رشت به صورت دستی ساخته شد و سعی کردم بر روی داده های آن به کمک بهترین الگوریتمی که در بخش قبل بدست آورده بودم ، ترینینگ انجام دهم. اما متاسفانه به دلیل تفاوت ابعادی این اتفاق نیافتاد و این بخش کد به نتیجه نرسید. اما اگر بخواهیم تحیلیل کنیم میتوانم به صورت زیر استدلال کنم :
بهترین مدلی که در بخش قبل بدست آوردیم پس از پر کردن داده های خالی بر اساس میانگین و مد داده های دیتاست قبلی بود. همچنین از انکدری استفاده شد که باز هم مختص داده های قبلی بود. همچنی از برای حذف فیچز ها و کاهش بعد از ماتریس کوریلیشنی استفاده کردیم که بازهم مختص داده های قبلی بود. یعنی برای داده های فعلی شاید بهتر باشد فیچر های بهتری حذف شوند تا به بهترین نتیجه برسیم. همچنین بهترین مدلی که ما در حالت قبل بدست آوردیم بر اساس ۱۰ ویژگی مهم دیتاست قبلی بود که ممکن است در دیتاست شهر رشت اهمیت زیادی نداشته باشند. برای مثال در تمامی سطر ها مقادیر ثابت داشته باشند یا اصلا جزو ده ویژگی مهم دیتاست شهر رشت نباشند. بنابر دلایل ذکر شده لزومی ندارد که بهترین مدلی که در حالت ثبل بدست آوردیم بر روی دیتاست شهر رشت نیز نتیجه خوبی داشته باشد و نمیتوانیم انتظار دقت بالایی از آن داشته باشیم